### 데이터셋 다운

In [1]:
!unzip -q "/content/drive/MyDrive/shared_drive/차량상태인식.zip" -d /content/dataset

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00


### 패키지 설치

In [4]:
import os
import glob
import cv2
import shutil
import pandas as pd

def prepare_yolo_dataset_by_csv(root_path, csv_path, output_path):
    # 1. CSV 파일 읽기 및 매핑 사전 생성
    # utf-8-sig는 파일 시작의 BOM(\ufeff)을 제거하기 위함입니다.
    df = pd.read_csv(csv_path, encoding='utf-8-sig')

    # 필요한 컬럼만 추출하고 유효한 split 값만 필터링
    df = df[['env', 'split']].dropna()
    df = df[df['split'].isin(['train', 'val', 'test'])]

    # env명을 key로, split(train/val/test)을 value로 하는 사전 생성
    split_map = dict(zip(df['env'], df['split']))
    print(f"✅ CSV 로드 완료: {len(split_map)}개의 환경(env) 설정 읽음.")

    # 2. 모든 png 파일 찾기
    # 데이터 구조에 맞춰 **/img/*.png 패턴으로 검색
    image_paths = glob.glob(os.path.join(root_path, "**/img/*.png"), recursive=True)
    print(f"총 {len(image_paths)}개의 이미지를 찾았습니다.")

    if len(image_paths) == 0:
        print("❌ 이미지를 찾지 못했습니다. 경로를 확인해주세요.")
        return

    # 3. 폴더 구조 생성 (train, val, test)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

    # 4. 데이터 처리 및 복사
    processed_count = 0
    missing_env_count = 0

    for img_path in image_paths:
        path_parts = img_path.split(os.sep)

        # 구조가 'folder/img/file.png' 일 때:
        # path_parts[-3] = 폴더명(env)
        folder_name = path_parts[-3]
        file_name = path_parts[-1]

        # CSV에 정의된 split 찾기
        target_split = split_map.get(folder_name)

        if not target_split:
            # CSV에 없는 폴더는 건너뜁니다.
            missing_env_count += 1
            continue

        # 새로운 파일명 생성: ArtHall_000000.png
        new_base_name = f"{folder_name}_{file_name}"
        new_label_name = new_base_name.replace('.png', '.txt')

        # 원본 라벨 경로 찾기 (img -> new_txt)
        org_label_path = img_path.replace('img', 'new_txt').replace('.png', '.txt')

        if not os.path.exists(org_label_path):
            continue

        # 이미지 로드 및 크기 확인
        img = cv2.imread(img_path)
        if img is None: continue
        h, w, _ = img.shape

        yolo_labels = []
        with open(org_label_path, 'r') as f:
            for line in f.readlines():
                data = list(map(float, line.split()))
                if len(data) < 5: continue

                # 0-3: 절대좌표(x1, y1, x2, y2), 4: Agent(Class)
                x1, y1, x2, y2 = data[0:4]
                class_id = int(data[4])

                # YOLO 정규화 변환
                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                nw = (x2 - x1) / w
                nh = (y2 - y1) / h

                yolo_labels.append(f"{class_id} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}")

        if yolo_labels:
            # 1. 이미지 복사 (지정된 split 폴더로)
            target_img_path = os.path.join(output_path, target_split, 'images', new_base_name)
            shutil.copy(img_path, target_img_path)

            # 2. YOLO 라벨 저장 (지정된 split 폴더로)
            target_label_path = os.path.join(output_path, target_split, 'labels', new_label_name)
            with open(target_label_path, 'w') as f:
                f.write("\n".join(yolo_labels))

            processed_count += 1

    print("-" * 30)
    print(f"✨ 처리 완료!")
    print(f"- 성공적으로 변환된 이미지: {processed_count}장")
    print(f"- CSV에 정의되지 않아 제외된 폴더: {missing_env_count}개")
    print(f"- 결과 저장소: {os.path.abspath(output_path)}")

# --- 실행 방법 ---
# root_path: 원본 데이터가 있는 곳 (예: dataset/train)
# csv_path: 제공해주신 csv 파일 경로
# output_path: YOLO 포맷으로 저장될 경로
# prepare_yolo_dataset_by_csv('./dataset/train', './split_assignment.csv', './yolo_final_data')

In [5]:
prepare_yolo_dataset_by_csv('./dataset/train', './split_assignment.csv', './yolo_final_data')

✅ CSV 로드 완료: 165개의 환경(env) 설정 읽음.
총 33187개의 이미지를 찾았습니다.
------------------------------
✨ 처리 완료!
- 성공적으로 변환된 이미지: 33187장
- CSV에 정의되지 않아 제외된 폴더: 0개
- 결과 저장소: /content/yolo_final_data


In [2]:
import os
import glob
import shutil
import pandas as pd
from tqdm import tqdm

def split_dataset_by_csv_raw(root_path, csv_path, output_path):
    # 1. CSV 파일 읽기 및 매핑 사전 생성
    # utf-8-sig는 파일 시작의 BOM(\ufeff)을 제거하기 위함입니다.
    df = pd.read_csv(csv_path, encoding='utf-8-sig')

    # 필요한 컬럼만 추출하고 유효한 split 값만 필터링
    df = df[['env', 'split']].dropna()
    df = df[df['split'].isin(['train', 'val', 'test'])]

    # env명을 key로, split(train/val/test)을 value로 하는 사전 생성
    split_map = dict(zip(df['env'], df['split']))
    print(f"✅ CSV 로드 완료: {len(split_map)}개의 환경(env) 설정 읽음.")

    # 2. 모든 png 파일 찾기
    image_paths = glob.glob(os.path.join(root_path, "**/img/*.png"), recursive=True)
    print(f"총 {len(image_paths)}개의 이미지를 찾았습니다.")

    if len(image_paths) == 0:
        print("❌ 이미지를 찾지 못했습니다. 경로를 확인해주세요.")
        return

    # 3. 폴더 구조 생성 (train/img, train/new_txt 등)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_path, split, 'img'), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, 'new_txt'), exist_ok=True)

    # 4. 데이터 처리 및 복사
    processed_count = 0
    missing_env_count = 0

    print("🚀 파일 복사 시작...")
    for img_path in tqdm(image_paths):
        path_parts = img_path.split(os.sep)

        # 구조: .../{folder_name}/img/{file_name}.png
        folder_name = path_parts[-3]
        file_name = path_parts[-1]

        # CSV에 정의된 split 찾기
        target_split = split_map.get(folder_name)

        if not target_split:
            missing_env_count += 1
            continue

        # 원본 라벨(txt) 경로 확인
        # img 폴더의 파일을 new_txt 폴더의 대응하는 파일로 변경
        org_label_path = img_path.replace(f'{os.sep}img{os.sep}', f'{os.sep}new_txt{os.sep}').replace('.png', '.txt')

        # 라벨 파일이 존재하는 경우에만 복사 진행
        if os.path.exists(org_label_path):
            # 새로운 파일명: ArtHall_000000.png / ArtHall_000000.txt
            new_base_name = f"{folder_name}_{file_name}"
            new_label_name = new_base_name.replace('.png', '.txt')

            # 대상 경로 설정
            target_img_dir = os.path.join(output_path, target_split, 'img')
            target_txt_dir = os.path.join(output_path, target_split, 'new_txt')

            # 파일 복사 (shutil.copy2는 메타데이터까지 보존)
            shutil.copy2(img_path, os.path.join(target_img_dir, new_base_name))
            shutil.copy2(org_label_path, os.path.join(target_txt_dir, new_label_name))

            processed_count += 1

    print("-" * 30)
    print(f"✨ 처리 완료!")
    print(f"- 성공적으로 복사된 데이터 쌍: {processed_count}개")
    print(f"- CSV에 정의되지 않아 제외된 폴더: {missing_env_count}개")
    print(f"- 결과 저장소: {os.path.abspath(output_path)}")

# --- 실행 방법 ---
# root_path: 원본 데이터셋 (ArtHall, Gapcheon 등 폴더들이 모여있는 곳)
# csv_path: split_assignment.csv 파일 경로
# output_path: 나누어진 결과물이 저장될 곳
# split_dataset_by_csv_raw('./dataset/train', './split_assignment.csv', './split_raw_data')

In [3]:
split_dataset_by_csv_raw('./dataset/train', './split_assignment.csv', './split_raw_data')

✅ CSV 로드 완료: 165개의 환경(env) 설정 읽음.
총 33187개의 이미지를 찾았습니다.
🚀 파일 복사 시작...


100%|██████████| 33187/33187 [05:31<00:00, 100.18it/s]

------------------------------
✨ 처리 완료!
- 성공적으로 복사된 데이터 쌍: 33187개
- CSV에 정의되지 않아 제외된 폴더: 0개
- 결과 저장소: /content/split_raw_data


In [ ]:
# --- Install ---
!pip install numpy filterpy scipy imageio tqdm
!git clone https://github.com/abewley/sort.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=55879d7b6d824356a3aed1e422ef55e9aa9170740ed272ee862423a37d21ed81
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy
Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 208 (delta 45), reused 40 (delta 40), pack-reused 159 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 22.84 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
%%writefile /content/sort_tracker.py
# =====================================================
# SORT: Simple Online and Realtime Tracking
# Source adapted from https://github.com/abewley/sort
# =====================================================

import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
        (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh
    )
    return o


class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array(
            [[1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]]
        )
        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

        self.kf.x[:4] = bbox.reshape((4,1))
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(bbox.reshape((4,1)))

    def predict(self):
        if (self.kf.x[6] + self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.kf.x)
        return self.kf.x


class Sort:
    def __init__(self, max_age=5, min_hits=2, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0,4))):
        self.frame_count += 1

        trks = np.zeros((len(self.trackers), 4))
        to_del = []
        for t, trk in enumerate(self.trackers):
            pos = trk.predict()
            trks[t] = pos[:4].reshape((1,4))
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            dets, trks, self.iou_threshold
        )

        for m in matched:
            self.trackers[m[1]].update(dets[m[0]])

        for i in unmatched_dets:
            self.trackers.append(KalmanBoxTracker(dets[i]))

        ret = []
        for trk in self.trackers:
            if trk.time_since_update < 1 and (trk.hits >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((trk.kf.x[:4].reshape((1,4)), [[trk.id]]), axis=1))
        return np.concatenate(ret) if len(ret) > 0 else np.empty((0,5))


def associate_detections_to_trackers(detections, trackers, iou_threshold):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0), dtype=int)

    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d,t] = iou(det, trk)

    matched_indices = linear_sum_assignment(-iou_matrix)
    matched_indices = np.array(list(zip(*matched_indices)))

    unmatched_detections = []
    for d in range(len(detections)):
        if d not in matched_indices[:,0]:
            unmatched_detections.append(d)

    unmatched_trackers = []
    for t in range(len(trackers)):
        if t not in matched_indices[:,1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))

    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

Writing /content/sort_tracker.py


In [ ]:
import sys
sys.path.insert(0, "/content")

from sort_tracker import Sort

tracker = Sort()
print("SORT import OK")

SORT import OK


In [ ]:
import cv2
import os
import glob
import numpy as np
import imageio
from collections import deque
from tqdm import tqdm

### code

In [ ]:
  class IntentAwareRiskManager:
    def __init__(self, window_size=5):
        self.window_size = window_size
        self.history = {}  # {track_id: deque[(cx, cy)]}

    def _update_history(self, track_id, center):
        if track_id not in self.history:
            self.history[track_id] = deque(maxlen=self.window_size)
        self.history[track_id].append(center)

    def _is_long_static(self, centers, move_thresh=3.0, min_static_frames=6):
        if len(centers) < min_static_frames + 1:
            return False

        diffs = [
            np.linalg.norm(np.array(centers[i]) - np.array(centers[i-1]))
            for i in range(1, len(centers))
        ]
        return all(d < move_thresh for d in diffs[-min_static_frames:])

    def _compute_2d_traj_metrics(self, centers, fps, img_w, img_h, eps=1e-3):
        """
        return: ttc, v_rel, d_curr
        """
        if len(centers) < 2:
            return float('inf'), 0.0, float('inf')

        ego = np.array([img_w / 2, img_h])
        p_prev = np.array(centers[-2])
        p_curr = np.array(centers[-1])

        d_prev = np.linalg.norm(p_prev - ego)
        d_curr = np.linalg.norm(p_curr - ego)
        v_rel = (d_prev - d_curr) * fps  # px/sec

        # 접근 안 하면 무효
        if v_rel < 5.0:
            return float('inf'), v_rel, d_curr

        # 장기 정지 필터
        if self._is_long_static(centers):
            return float('inf'), v_rel, d_curr

        ttc = d_curr / max(v_rel, eps)
        return ttc, v_rel, d_curr

    def calculate_risk(self, track_id, bbox, location, actions, fps, img_w, img_h):
        cx = (bbox[0] + bbox[2]) / 2
        cy = (bbox[1] + bbox[3]) / 2
        self._update_history(track_id, (cx, cy))
        centers = self.history[track_id]

        # --- Step 1: Physical risk ---
        ttc, v_rel, d_curr = self._compute_2d_traj_metrics(
            centers, fps, img_w, img_h
        )

        base_score = 0.0
        if ttc < 5.0:
            base_score = min(50.0, 50.0 * (3.0 / max(ttc, 0.5)))

        # --- Step 2: Semantic reasoning ---
        semantic_bonus = 0.0
        omega = 1.0
        reason = "Normal"

        # Ignore irrelevant zones
        if location in [2, 3, 4]:
            return {
                "score": 0.0,
                "ttc": "Safe",
                "reason": "Ignored",
                "level": "SAFE"
            }

        HIGH_SPEED = 80.0  # px/sec

        # Ego lane
        if location == 0:
            if ttc < 2.0 and v_rel > HIGH_SPEED:
                reason = "Rapid Approach"
                semantic_bonus = 10.0
            elif ttc < 2.0:
                reason = "Close Following"
            else:
                reason = "Normal Deceleration"

        # Adjacent lane (cut-in)
        elif location == 1:
            if actions[1] == 1 or actions[2] == 1:
                reason = "Dangerous Cut-in"
                semantic_bonus = 10.0
                omega = 1.3
            else:
                reason = "Approaching"
                omega = 0.8

        final_risk = min(base_score * omega + semantic_bonus, 100.0)

        return {
            "score": round(final_risk, 2),
            "ttc": round(ttc, 2) if ttc != float('inf') else "Safe",
            "reason": reason,
            "level": self._get_risk_level(final_risk),
        }

    def _get_risk_level(self, score):
        if score > 60: return "DANGER"
        if score > 30: return "CAUTION"
        return "SAFE"


In [ ]:
import cv2
import os
import glob
import numpy as np
import imageio
from tqdm import tqdm

def export_intent_risk_video(folder_path, output_name="Final_2D_Trajectory_Risk.mp4"):
    tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.3)
    risk_mgr = IntentAwareRiskManager(window_size=5)

    img_files = sorted(glob.glob(os.path.join(folder_path, "img/*.png")))
    if not img_files:
        print("❌ 이미지 경로 오류")
        return

    sample = cv2.imread(img_files[0])
    img_h, img_w, _ = sample.shape

    writer = imageio.get_writer(output_name, fps=1, codec="libx264", quality=8)
    print("🎬 Risk video generating...")

    # Risk level → bounding box color
    RISK_COLOR = {
        "DANGER":  (0, 0, 255),     # Red
        "WARNING": (0, 165, 255),   # Orange
        "CAUTION": (0, 255, 255),   # Yellow
        "SAFE":    (255, 0, 0)      # Blue
    }

    # Location text
    LOC_LABEL = {
        0: "Ego",
        1: "Adjacent",
        2: "Opposite",
        3: "Intersection",
        4: "Parking"
    }

    # Action text color
    ACTION_TEXT_COLOR = {
        "BRAKE":     (0, 0, 255),
        "LEFT":      (255, 255, 0),
        "RIGHT":     (0, 255, 255),
        "EMERGENCY": (255, 0, 255)
    }

    def actions_to_text(actions):
        texts = []
        if actions[0] == 1: texts.append("BRAKE")
        if actions[1] == 1: texts.append("LEFT")
        if actions[2] == 1: texts.append("RIGHT")
        if actions[3] == 1: texts.append("EMERGENCY")
        return texts

    for img_p in tqdm(img_files):
        frame = cv2.imread(img_p)
        label_p = img_p.replace("img", "new_txt").replace(".png", ".txt")

        dets, meta = [], []

        if os.path.exists(label_p):
            with open(label_p) as f:
                for line in f:
                    d = list(map(float, line.split()))
                    dets.append(d[0:4])
                    meta.append({
                        "location": int(d[5]),
                        "actions": d[6:10]
                    })

        if len(dets) > 0:
            tracks = tracker.update(np.array(dets))

            for x1, y1, x2, y2, tid in tracks:
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                bbox = [x1, y1, x2, y2]

                # detection ↔ track 매칭 (nearest x)
                i = np.argmin([abs(x1 - d[0]) for d in dets])
                location = meta[i]["location"]
                actions = meta[i]["actions"]

                res = risk_mgr.calculate_risk(
                    track_id=int(tid),
                    bbox=bbox,
                    location=location,
                    actions=actions,
                    fps=20,
                    img_w=img_w,
                    img_h=img_h
                )

                level = res["level"]
                color = RISK_COLOR[level]
                loc_text = LOC_LABEL.get(location, "Unknown")
                action_texts = actions_to_text(actions)

                thickness = 3 if level == "DANGER" else \
                            3 if level in ["WARNING", "CAUTION"] else 2

                # ===============================
                # 1. Bounding Box
                # ===============================
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

                # ===============================
                # 2. Text Rendering
                # ===============================
                if level == "SAFE":
                    cv2.putText(frame,
                                loc_text,
                                (x1, y1 - 20),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.45,
                                (200, 200, 200),
                                1)

                    y_action = y1 - 4
                    for act in action_texts:
                        cv2.putText(frame,
                                    f"{act}",
                                    (x1, y_action),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    0.45,
                                    ACTION_TEXT_COLOR[act],
                                    1)
                        y_action += 14

                else:
                    cv2.putText(frame,
                                f"[{level}] {res['score']} TTC:{res['ttc']}s",
                                (x1, y1 - 68),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.5,
                                color,
                                2)

                    cv2.putText(frame,
                                f"Cause: {res['reason']}",
                                (x1, y1 - 52),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.45,
                                (255, 255, 255),
                                1)

                    cv2.putText(frame,
                                loc_text,
                                (x1, y1 - 36),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.45,
                                (200, 200, 200),
                                1)

                    y_action = y1 - 20
                    for act in action_texts:
                        cv2.putText(frame,
                                    f"{act}",
                                    (x1, y_action),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    0.45,
                                    ACTION_TEXT_COLOR[act],
                                    1)
                        y_action += 14

                    if "duration" in res:
                        cv2.putText(frame,
                                    f"Duration: {res['duration']}s",
                                    (x1, y2 + 14),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    0.45,
                                    color,
                                    1)

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"✅ Saved: {output_name}")


In [ ]:
from glob import glob
from pathlib import Path
folder = Path("/content/split_raw_data/test/ETRINear_Fog")

TypeError: object of type 'PosixPath' has no len()

In [ ]:
TEST_FOLDER = "/content/dataset/train/ETRINear_Fog"
export_intent_risk_video(TEST_FOLDER, "ETRINear_Fog_result.mp4")

🎬 Risk video generating...


100%|██████████| 1297/1297 [04:56<00:00,  4.38it/s]


✅ Saved: ETRINear_Fog_result.mp4


In [ ]:
import shutil
import os

src = "/content/2D_Trajectory_Risk_5.mp4"
dst_dir = "/content/drive/MyDrive/risk_results"
os.makedirs(dst_dir, exist_ok=True)

dst = os.path.join(dst_dir, os.path.basename(src))
shutil.copy2(src, dst)

print(f"✅ Moved to {dst}")


✅ Moved to /content/drive/MyDrive/risk_results/2D_Trajectory_Risk_5.mp4


In [4]:
import os
import glob
import shutil
from tqdm import tqdm

def reorganize_dataset_to_folders(base_path):
    """
    /content/split_raw_data/test 경로를 입력받아
    내부의 img, new_txt 파일들을 각각의 시나리오 폴더로 다시 묶어줍니다.
    """
    img_source_dir = os.path.join(base_path, 'img')
    txt_source_dir = os.path.join(base_path, 'new_txt')

    # 1. 모든 이미지 파일 목록 가져오기
    img_files = glob.glob(os.path.join(img_source_dir, "*.png"))

    print(f"📦 총 {len(img_files)}개의 파일을 재정렬합니다...")

    for img_path in tqdm(img_files):
        # 파일명만 추출 (예: ETRINear_Fog_00003427.png)
        full_file_name = os.path.basename(img_path)

        # 파일명에서 폴더명과 실제 파일명 분리
        # (마지막 '_'를 기준으로 나눔 - 원본 파일명이 000000.png 형태이므로)
        last_underscore_idx = full_file_name.rfind('_')
        if last_underscore_idx == -1:
            continue

        folder_name = full_file_name[:last_underscore_idx]
        actual_file_name = full_file_name[last_underscore_idx + 1:]

        # 2. 새로운 폴더 구조 생성 (예: test/ETRINear_Fog/img)
        new_img_dir = os.path.join(base_path, folder_name, 'img')
        new_txt_dir = os.path.join(base_path, folder_name, 'new_txt')

        os.makedirs(new_img_dir, exist_ok=True)
        os.makedirs(new_txt_dir, exist_ok=True)

        # 3. 이미지 파일 이동
        target_img_path = os.path.join(new_img_dir, actual_file_name)
        shutil.move(img_path, target_img_path)

        # 4. 대응하는 텍스트 파일 이동
        # 이미지 파일명과 동일한 이름의 .txt 파일을 찾음
        txt_file_name = full_file_name.replace('.png', '.txt')
        old_txt_path = os.path.join(txt_source_dir, txt_file_name)

        if os.path.exists(old_txt_path):
            actual_txt_name = actual_file_name.replace('.png', '.txt')
            target_txt_path = os.path.join(new_txt_dir, actual_txt_name)
            shutil.move(old_txt_path, target_txt_path)

    # 5. 비어있는 원래의 img, new_txt 폴더 삭제 (선택 사항)
    if not os.listdir(img_source_dir):
        os.rmdir(img_source_dir)
    if not os.listdir(txt_source_dir):
        os.rmdir(txt_source_dir)

    print(f"✨ 재정렬 완료! {base_path} 내부를 확인하세요.")

In [5]:
TEST_BASE_PATH = '/content/split_raw_data/test'
reorganize_dataset_to_folders(TEST_BASE_PATH)

📦 총 3740개의 파일을 재정렬합니다...


100%|██████████| 3740/3740 [00:00<00:00, 13525.30it/s]

✨ 재정렬 완료! /content/split_raw_data/test 내부를 확인하세요.
